## News Articles Classification using NLP

In [1]:
## import dependencies

import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

## Spacy model

In [4]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [5]:
data = pd.read_csv('BBC News.csv')
data.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [6]:
data.isna().sum()

ArticleId    0
Text         0
Category     0
dtype: int64

In [7]:
data.Category.value_counts()

sport            346
business         336
politics         274
entertainment    273
tech             261
Name: Category, dtype: int64

In [8]:
data.shape

(1490, 3)

In [9]:
data.ArticleId.nunique()

1490

In [10]:
## drop ArticleId column
df = data.drop('ArticleId', axis=1)

### Preprocessing using Spacy

In [11]:
def is_whitespace(data):
    
    blank = []
    for idx, text, label in data.itertuples():
        if text.isspace():
            blank.append(idx)
    
    return blank

In [12]:
is_whitespace(df)

[]

In [13]:
 # remove stop words and lemmatize the text

def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct or token.is_space:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens)

In [14]:
df['processed_text'] = df['Text'].apply(preprocess)

In [15]:
df.head()

,Text,Category,processed_text
0,worldcom ex-boss launches defence lawyers defe...,business,worldcom ex boss launch defence lawyer defend ...
1,german business confidence slides german busin...,business,german business confidence slide german busine...
2,bbc poll indicates economic gloom citizens in ...,business,bbc poll indicate economic gloom citizen major...
3,lifestyle governs mobile choice faster bett...,tech,lifestyle govern mobile choice fast well funky...
4,enron bosses in $168m payout eighteen former e...,business,enron boss $ 168 m payout eighteen enron direc...


In [16]:
df.Text[0]

'worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr ebb

In [17]:
df.processed_text[0]

'worldcom ex boss launch defence lawyer defend worldcom chief bernie ebber battery fraud charge call company whistleblow witness cynthia cooper worldcom s ex head internal accounting alert director irregular accounting practice telecoms giant 2002 warning lead collapse firm follow discovery $ 11bn £ 5.7bn accounting fraud mr ebber plead guilty charge fraud conspiracy prosecution lawyer argue mr ebber orchestrate series accounting trick worldcom order employee hide expense inflate revenue meet wall street earning estimate ms cooper run consulting business tell jury new york wednesday external auditor arthur andersen approve worldcom s accounting early 2001 2002 say andersen give green light procedure practice worldcom mr ebber s lawyer say unaware fraud argue auditor alert problem ms cooper say shareholder meeting mr ebber pass technical question company s finance chief give brief answer prosecution s star witness worldcom financial chief scott sullivan say mr ebber order accounting adj

#### Label Encoding and Data Splitting

In [18]:
le = LabelEncoder()
cat_fit = le.fit(df.Category)
y = cat_fit.transform(df.Category)

In [19]:
# Train-Test split

X_train, X_test, y_train, y_test = train_test_split(df.processed_text, y,test_size=0.2, random_state=42)

## **Data** **Modelling**

### CountVectorizer

In [20]:
model1 = Pipeline([('c_vectorizer', CountVectorizer(ngram_range=(1, 2))), 
                      ('bayes_model', MultinomialNB())])

In [21]:
model1.fit(X_train, y_train)

Pipeline(steps=[('c_vectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('bayes_model', MultinomialNB())])

In [22]:
model1_pred = model1.predict(X_test)

In [23]:
print(f'Accuracy score of count_vectorizer based model: {accuracy_score(y_test, model1_pred):.2f}')

Accuracy score of count_vectorizer based model: 0.98


## TF-IDF Vectorizer

In [24]:
model2 = Pipeline([('t_vector', TfidfVectorizer()), 
                    ('bayes_model_2', MultinomialNB())])

In [25]:
model2.fit(X_train, y_train)

Pipeline(steps=[('t_vector', TfidfVectorizer()),
                ('bayes_model_2', MultinomialNB())])

In [26]:
model2_pred = model2.predict(X_test)

In [27]:
print(f'Accuracy score of tfidf based model: {accuracy_score(y_test, model2_pred):.2f}')

Accuracy score of tfidf based model: 0.96
